# Plugin Interface

> Abstract base class defining the generic plugin interface

In [ ]:
#| default_exp core.interface

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
from abc import ABC, abstractmethod
import logging
from typing import Optional, Dict, Any, Tuple, Generator
from cjm_plugin_system.utils.validation import validate_config, extract_defaults

## PluginInterface

The `PluginInterface` is a generic abstract base class that defines the contract all plugins must implement. This interface is completely domain-agnostic and can be subclassed to create specific plugin systems (e.g., transcription plugins, LLM plugins, image generation plugins).

In [ ]:
#| export
class PluginInterface(ABC):
    """Generic plugin interface that all plugins must implement.
    
    This is a domain-agnostic base class. Domain-specific plugin systems
    should subclass this interface and add their specific requirements.
    
    Example:
        >>> class TranscriptionPlugin(PluginInterface):
        ...     @property
        ...     @abstractmethod
        ...     def supported_formats(self) -> List[str]:
        ...         '''Audio formats this plugin supports.'''
        ...         pass
        ...
        ...     @abstractmethod
        ...     def execute(
        ...         self,
        ...         audio: Union[AudioData, str, Path],
        ...         **kwargs
        ...     ) -> TranscriptionResult:
        ...         '''Transcribe audio to text.'''
        ...         pass
    """
    
    @property
    @abstractmethod
    def name(
        self
    ) -> str:  # The unique identifier for this plugin
        """Unique plugin identifier."""
        pass
    
    @property
    @abstractmethod
    def version(
        self
    ) -> str:  # The semantic version string (e.g., "1.0.0")
        """Plugin version."""
        pass
    
    @abstractmethod
    def initialize(
        self,
        config: Optional[Dict[str, Any]] = None  # Configuration dictionary for plugin-specific settings
    ) -> None:
        """Initialize the plugin with configuration.
        
        Args:
            config: Optional configuration dictionary. If not provided,
                   defaults from the schema will be used.
        """
        pass
    
    @abstractmethod
    def execute(
        self,
        *args,
        **kwargs
    ) -> Any:  # Returns plugin-specific output
        """Execute the plugin's main functionality.
        
        The signature and return type of this method should be defined
        by domain-specific plugin interfaces.
        """
        pass
    
    @abstractmethod
    def is_available(
        self
    ) -> bool:  # True if all required dependencies are available
        """Check if the plugin's dependencies are available.
        
        Returns:
            True if the plugin can be used, False if dependencies are missing.
        """
        pass

    @staticmethod
    @abstractmethod
    def get_config_schema(
    ) -> Dict[str, Any]:  # JSON Schema describing configuration options
        """Return JSON Schema describing the plugin's configuration options.
        
        The schema should follow JSON Schema Draft 7 specification.
        This enables automatic UI generation and validation.
        
        Returns:
            Dictionary containing a JSON Schema
            
        Example:
            >>> @staticmethod
            >>> def get_config_schema() -> Dict[str, Any]:
            ...     return {
            ...         "type": "object",
            ...         "properties": {
            ...             "model": {
            ...                 "type": "string",
            ...                 "enum": ["tiny", "base", "small"],
            ...                 "default": "base",
            ...                 "description": "Model size to use"
            ...             }
            ...         },
            ...         "required": ["model"]
            ...     }
        """
        pass
    
    @abstractmethod
    def get_current_config(
        self
    ) -> Dict[str, Any]:  # Current configuration state
        """Return the current configuration state.
        
        This should return the actual configuration being used by the plugin,
        which may include defaults not explicitly set by the user.
        
        Returns:
            Dictionary containing current configuration values
        """
        pass
    
    def validate_config(
        self,
        config: Dict[str, Any]  # Configuration to validate
    ) -> Tuple[bool, Optional[str]]:  # (is_valid, error_message)
        """Validate a configuration dictionary against the schema.
        
        Uses the plugin's schema from `get_config_schema()` to validate
        the provided configuration.
        
        Args:
            config: Configuration dictionary to validate
            
        Returns:
            Tuple of (is_valid, error_message).
            If valid, error_message is None.
        """
        schema = self.get_config_schema()
        return validate_config(config, schema)
    
    def get_config_defaults(
        self
    ) -> Dict[str, Any]:  # Default values from schema
        """Extract default values from the configuration schema.
        
        Returns a dictionary of default values for all properties
        that have defaults defined in the schema.
        
        Returns:
            Dictionary of default configuration values
        """
        schema = self.get_config_schema()
        return extract_defaults(schema)
    
    def cleanup(
        self
    ) -> None:
        """Optional cleanup when plugin is unloaded.
        
        Override this method to perform any necessary cleanup,
        such as releasing resources or closing connections.
        """
        pass

## Streaming Support

The plugin system includes optional streaming support. Plugins can implement `execute_stream()` to provide real-time streaming results.

In [ ]:
#| export
def PluginInterface_supports_streaming(self) -> bool:
    """Check if this plugin supports streaming execution.
    
    Returns:
        True if execute_stream is implemented and streaming is supported
    """
    # Default: check if execute_stream is overridden from the base class
    return type(self).execute_stream != PluginInterface.execute_stream

def PluginInterface_execute_stream(
    self,
    *args,  # Arguments for plugin execution
    **kwargs  # Keyword arguments for plugin execution
) -> Generator[Any, None, Any]:  # Yields partial results, returns final result
    """Stream execution results chunk by chunk.
    
    Default implementation falls back to execute() without streaming.
    Plugins can override this to provide real streaming capabilities.
    
    Args:
        *args: Positional arguments to pass to the plugin
        **kwargs: Keyword arguments to pass to the plugin
        
    Yields:
        Partial results as they become available (type depends on plugin)
        
    Returns:
        Final complete result
        
    Example:
        >>> # Stream results in real-time
        >>> for chunk in plugin.execute_stream(input_data):
        ...     print(chunk, end="", flush=True)
        >>> 
        >>> # Or collect all chunks and get final result
        >>> generator = plugin.execute_stream(input_data)
        >>> chunks = []
        >>> for chunk in generator:
        ...     chunks.append(chunk)
    """
    # Default implementation: execute normally and yield complete result at once
    result = self.execute(*args, **kwargs)
    yield result
    return result

# Add the methods to the PluginInterface class
PluginInterface.supports_streaming = PluginInterface_supports_streaming
PluginInterface.execute_stream = PluginInterface_execute_stream

### Example: Creating a Plugin

Here's an example of how to create a concrete plugin implementation:

In [ ]:
import logging

class ExamplePlugin(PluginInterface):
    """An example plugin implementation with configuration schema."""

    def __init__(self):
        self.logger = logging.getLogger(f"{__name__}.{type(self).__name__}")
        self.config = {}
        self.resource = None
    
    @property
    def name(self) -> str:
        return "example_plugin"
    
    @property
    def version(self) -> str:
        return "1.0.0"

    @staticmethod
    def get_config_schema() -> Dict[str, Any]:
        """Return the configuration schema for this plugin."""
        return {
            "type": "object",
            "properties": {
                "mode": {
                    "type": "string",
                    "enum": ["fast", "balanced", "quality"],
                    "default": "balanced",
                    "description": "Processing mode"
                },
                "threshold": {
                    "type": "number",
                    "minimum": 0.0,
                    "maximum": 1.0,
                    "default": 0.5,
                    "description": "Processing threshold"
                },
                "max_workers": {
                    "type": "integer",
                    "minimum": 1,
                    "maximum": 16,
                    "default": 4,
                    "description": "Maximum number of workers"
                },
                "enable_cache": {
                    "type": "boolean",
                    "default": True,
                    "description": "Enable result caching"
                }
            },
            "required": ["mode"]
        }
    
    def get_current_config(self) -> Dict[str, Any]:
        """Return the current configuration."""
        # Merge defaults with actual config
        defaults = self.get_config_defaults()
        return {**defaults, **self.config}
    
    def initialize(self, config: Optional[Dict[str, Any]] = None) -> None:
        """Initialize the plugin."""
        if config:
            is_valid, error = self.validate_config(config)
            if not is_valid:
                raise ValueError(f"Invalid configuration: {error}")
        
        # Merge provided config with defaults
        defaults = self.get_config_defaults()
        self.config = {**defaults, **(config or {})}
        
        self.logger.info(f"Initializing {self.name} with config: {self.config}")
        
        # Simulate resource initialization
        mode = self.config.get("mode", "balanced")
        self.resource = f"Resource-{mode}"
    
    def execute(self, input_data: Any, **kwargs) -> Any:
        """Execute the plugin's functionality."""
        self.logger.info(f"Executing {self.name} with resource: {self.resource}")
        self.logger.info(f"Config: {self.config}")
        return f"Processed {input_data} using {self.resource}"

    def is_available(self) -> bool:
        """Check availability."""
        return True
    
    def cleanup(self) -> None:
        """Clean up resources."""
        self.logger.info(f"Cleaning up {self.name}")
        self.resource = None

In [ ]:
# Test the example plugin
logging.basicConfig(level=logging.INFO)

plugin = ExamplePlugin()
print(f"Plugin: {plugin.name} v{plugin.version}")
print(f"Available: {plugin.is_available()}")

# Get schema and defaults
import json
schema = plugin.get_config_schema()
print("\nConfiguration Schema:")
print(json.dumps(schema, indent=2))

defaults = plugin.get_config_defaults()
print("\nDefault Configuration:")
print(json.dumps(defaults, indent=2))

Plugin: example_plugin v1.0.0
Available: True

Configuration Schema:
{
  "type": "object",
  "properties": {
    "mode": {
      "type": "string",
      "enum": [
        "fast",
        "balanced",
        "quality"
      ],
      "default": "balanced",
      "description": "Processing mode"
    },
    "threshold": {
      "type": "number",
      "minimum": 0.0,
      "maximum": 1.0,
      "default": 0.5,
      "description": "Processing threshold"
    },
    "max_workers": {
      "type": "integer",
      "minimum": 1,
      "maximum": 16,
      "default": 4,
      "description": "Maximum number of workers"
    },
    "enable_cache": {
      "type": "boolean",
      "default": true,
      "description": "Enable result caching"
    }
  },
  "required": [
    "mode"
  ]
}

Default Configuration:
{
  "mode": "balanced",
  "threshold": 0.5,
  "max_workers": 4,
  "enable_cache": true
}


In [ ]:
# Initialize and execute
plugin.initialize({"mode": "quality", "threshold": 0.8})
result = plugin.execute("sample_data")
print(f"\nResult: {result}")

# Get current config
current = plugin.get_current_config()
print("\nCurrent Configuration:")
print(json.dumps(current, indent=2))

# Cleanup
plugin.cleanup()

INFO:__main__.ExamplePlugin:Initializing example_plugin with config: {'mode': 'quality', 'threshold': 0.8, 'max_workers': 4, 'enable_cache': True}
INFO:__main__.ExamplePlugin:Executing example_plugin with resource: Resource-quality
INFO:__main__.ExamplePlugin:Config: {'mode': 'quality', 'threshold': 0.8, 'max_workers': 4, 'enable_cache': True}
INFO:__main__.ExamplePlugin:Cleaning up example_plugin



Result: Processed sample_data using Resource-quality

Current Configuration:
{
  "mode": "quality",
  "threshold": 0.8,
  "max_workers": 4,
  "enable_cache": true
}


In [ ]:
# Test configuration validation
test_configs = [
    ({"mode": "fast"}, "Valid minimal config"),
    ({"mode": "invalid"}, "Invalid mode value"),
    ({"threshold": 0.5}, "Missing required 'mode' field"),
    ({"mode": "balanced", "threshold": 2.0}, "Threshold exceeds maximum"),
]

print("\nConfiguration Validation Tests:")
for config, description in test_configs:
    is_valid, error = plugin.validate_config(config)
    print(f"\n{description}:")
    print(f"  Config: {config}")
    print(f"  Valid: {is_valid}")
    if error:
        print(f"  Error: {error[:80]}...")  # Truncate long errors


Configuration Validation Tests:

Valid minimal config:
  Config: {'mode': 'fast'}
  Valid: True

Invalid mode value:
  Config: {'mode': 'invalid'}
  Valid: False
  Error: 'invalid' is not one of ['fast', 'balanced', 'quality']

Failed validating 'enum...

Missing required 'mode' field:
  Config: {'threshold': 0.5}
  Valid: False
  Error: 'mode' is a required property

Failed validating 'required' in schema:
    {'typ...

Threshold exceeds maximum:
  Config: {'mode': 'balanced', 'threshold': 2.0}
  Valid: False
  Error: 2.0 is greater than the maximum of 1.0

Failed validating 'maximum' in schema['p...


In [ ]:
# Test streaming support
print(f"\nSupports streaming: {plugin.supports_streaming()}")

# The default implementation doesn't stream, it just yields the result once
print("\nStreaming execution:")
for chunk in plugin.execute_stream("stream_data"):
    print(f"  Chunk: {chunk}")

INFO:__main__.ExamplePlugin:Executing example_plugin with resource: None
INFO:__main__.ExamplePlugin:Config: {'mode': 'quality', 'threshold': 0.8, 'max_workers': 4, 'enable_cache': True}



Supports streaming: False

Streaming execution:
  Chunk: Processed stream_data using None


In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()